In [1]:
import numpy  as np
import pandas as pd
import fnmatch
import os
import re

import numpy  as np
import matplotlib.pyplot as plt
from os.path import splitext as ossplitext


In [2]:
def find_files(path_main, pattern, type='separate'):
	list_return = list()
	for dirpath, dirs, files in os.walk(path_main):
		for fname in fnmatch.filter(files, pattern):
			list_return.append((dirpath,fname))

	if type == 'separate':
		list_return = np.asarray(list_return)
		df 			= pd.DataFrame(list_return, columns = ['path','file'])
		df 			= df.sort_values(by=['path'], ascending=True)
		return df.to_numpy()

	if type == 'absolute':
		new_list = list()
		for i,j in list_return:
			new_list.append(os.path.join(i,j))
		new_list = sorted(new_list )
		return np.asarray(new_list)
	else:
		print('error, you need choise type: [separate,absolute]')
		return None

def find_dirs(path_main, pattern, type='separate'):
	list_return = list()
	for dirpath, dirs, files in os.walk(path_main):
		for dname in fnmatch.filter(dirs, pattern):
			list_return.append((dirpath,dname))
	if type == 'separate':
		list_return = np.asarray(list_return)
		df 			= pd.DataFrame(list_return, columns = ['path','dir'])
		# df 			= df.sort_values(by=['path'], ascending=True)
		return df.to_numpy()

	if type == 'absolute':
		new_list = list()
		for i,j in list_return:
			new_list.append(os.path.join(i,j))
		# new_list = sorted(new_list )
		return np.asarray(new_list)
	else:
		print('error, you need choise type: [separate,absolute]')
		return None

def linspace(start, end, step=1.):
    assert start<end, "start need greater than end"
#     epsilon = 0.000001
#     result = np.arange(start,end+epsilon,step)
    result = np.arange(start,end,step)
    result =  np.around(result, decimals=3)
    result = list(result)
    if result[-1]!=end:
        result.append(end)     
    return result


In [3]:

def build_dataframe(files):
    cols = ['queryAbsPath','qId','qIsPesentGT','beta','eta','tau','highSimilarity','qIsIn_eta','type']
    main_df = pd.DataFrame(columns=cols)

    for i in files:
        df = pd.read_csv(i, usecols=cols)
        if df.shape[0]>0:
            main_df = main_df.append(df, ignore_index=True)
    
    return main_df

In [4]:

def build_outcome_dataframe(files):
#     cols = ['imgAbsPath','cosine_similarity']
    cols = ['imgAbsPath_origin','imgAbsPath_current','cosine_similarity']
    main_df = pd.DataFrame(columns=cols)

    for i in files:
        df = pd.read_csv(i, usecols=cols)
        if df.shape[0]>0:
            main_df = main_df.append(df, ignore_index=True)
    return main_df

In [5]:
def create_dir(path):
    try:
        os.makedirs(path, exist_ok=True)
        # print('{} created successfully'.format(path))
        return path
    except:
        print('{} cant be created')
    return None

In [6]:
xcols    = ['imgAbsPath_origin','imgAbsPath_current','cosine_similarity']
xmain_df = pd.DataFrame(columns=xcols)
xmain_df.shape

(0, 3)

# PATHs

In [7]:
path_main   = './../dataset_prid2011/1_Skip'


In [8]:
path_save   = './results/1_Skip'

In [9]:
create_dir(path_save)

'./results/1_Skip'

In [10]:
def gather_outcome_dataframes(path_main, path_save, prefix_reid, max_tracklet, thisCam=False, otherCam=True):
#     if thisCam:
#         dirs_outcomes = find_dirs(path_main, 'max_tracklet_{}_tau_frameStart_*'.format(max_tracklet), type='absolute')
#         list_tmp      = list()
#         for dirs in dirs_outcomes:
#             tmp_outcomefilesThisCam = find_files(str(dirs), '{}_outcome_thisCam_person_*_dataframe.csv'.format(prefix_reid), type='absolute')
#             for j in tmp_outcomefilesThisCam:
#                 list_tmp.append(j)
#         outcomefilesThisCam = np.asarray(list_tmp)
#         df_outcome_ThisCam  = build_outcome_dataframe(outcomefilesThisCam)
#         new_path_save        = create_dir(os.path.join(path_save, 'Skip_max_tracklet_{}'.format(max_tracklet)))
#         df_outcome_ThisCam.to_csv(os.path.join(new_path_save,'Skip_df_outcomes_thisCam_{}.csv'.format(prefix_reid)), index=False, header=True)
    
    if otherCam:
        dirs_outcomes = find_dirs(path_main, 'max_tracklet_{}_tau_frameStart_*'.format(max_tracklet), type='absolute')
        list_tmp      = list()

        for dirs in dirs_outcomes:
            tmp_outcomefilesOtherCam = find_files(str(dirs), '{}_outcome_otherCam_person_*_dataframe.csv'.format(prefix_reid), type='absolute')
            for j in tmp_outcomefilesOtherCam:
                list_tmp.append(j)

        outcomefilesOtherCam = np.asarray(list_tmp)
        df_outcome_otherCam  = build_outcome_dataframe(outcomefilesOtherCam)
        new_path_save        = create_dir(os.path.join(path_save, 'Skip_max-tracklet-{}'.format(max_tracklet)))
        df_outcome_otherCam.to_csv(os.path.join(new_path_save,'Skip_{}_gather-df-outcomes_otherCam.csv'.format(prefix_reid)), index=False, header=True)

# READ  and GATHER of Outcomes Re-id.

In [11]:
prefix_reid = 'SiamIDL'

In [12]:
max_tracklets = [1,5,10,20,40,80]
for i in max_tracklets:
    gather_outcome_dataframes(path_main, path_save, prefix_reid, i)


In [13]:
prefix_reid = 'BoT'

In [14]:
max_tracklets = [1,5,10,20,40,80]
for i in max_tracklets:
    gather_outcome_dataframes(path_main, path_save, prefix_reid, i)


# READ and GATHER of resultGUI

In [15]:
def gather_resultsGUI(path_main, path_save, prefix_reid, max_tracklet, thisCam=False, otherCam=True):
#     if thisCam:
#         dirs_outcomes = find_dirs(path_main, 'max_tracklet_{}_tau_frameStart_*'.format(max_tracklet), type='absolute')
#         list_tmp      = list()
#         for dirs in dirs_outcomes:
#             tmp_filesThisCam = find_files(str(dirs), '{}_resultGUI_thisCam_person_*_seqVideo_*.csv'.format(prefix_reid), type='absolute')
#             for j in tmp_filesThisCam:
#                 list_tmp.append(j)
#         outcomefilesThisCam = np.asarray(list_tmp)
#         df_ThisCam          = build_dataframe(outcomefilesThisCam)
#         new_path_save        = create_dir(os.path.join(path_save, 'Skip_max-tracklet_{}'.format(max_tracklet)))
#         df_ThisCam.to_csv(os.path.join(new_path_save,'Skip_{}_df_resultGUI_thisCam.csv'.format(prefix_reid)), index=False, header=True)
    
    if otherCam:
        dirs_outcomes = find_dirs(path_main, 'max_tracklet_{}_tau_frameStart_*'.format(max_tracklet), type='absolute')
        list_tmp      = list()
        for dirs in dirs_outcomes:
            tmp_filesOtherCam = find_files(str(dirs), '{}_resultGUI_otherCam_person_*_seqVideo_*.csv'.format(prefix_reid), type='absolute')
            for j in tmp_filesOtherCam:
                list_tmp.append(j)
        outcomefilesOtherCam = np.asarray(list_tmp)
        df_otherCam          = build_dataframe(outcomefilesOtherCam)
        new_path_save        = create_dir(os.path.join(path_save, 'Skip_max-tracklet-{}'.format(max_tracklet)))
        df_otherCam.to_csv(os.path.join(new_path_save,'Skip_{}_gather-df-resultGUI_otherCam.csv'.format(prefix_reid)), index=False, header=True)

In [16]:
prefix_reid = 'SiamIDL'

In [17]:
max_tracklets = [1,5,10,20,40,80]
for i in max_tracklets:
    gather_resultsGUI(path_main, path_save, prefix_reid, i)


In [18]:
prefix_reid = 'BoT'

In [19]:
max_tracklets = [1,5,10,20,40,80]
for i in max_tracklets:
    gather_resultsGUI(path_main, path_save, prefix_reid, i)
